In [ ]:
pip install ortools

In [ ]:
#task 1

from ortools.sat.python import cp_model

def solve_sudoku(grid):
    model = cp_model.CpModel()
    size = 9
    subgrid_size = 3
    cells = [[model.NewIntVar(1, size, f"cell_{r}_{c}") for c in range(size)] for r in range(size)]

    for i in range(size):
        model.AddAllDifferent(cells[i])
        model.AddAllDifferent([cells[r][i] for r in range(size)])

    for r in range(0, size, subgrid_size):
        for c in range(0, size, subgrid_size):
            model.AddAllDifferent([cells[r + dr][c + dc] for dr in range(subgrid_size) for dc in range(subgrid_size)])

    for i in range(size):
        model.Add(sum(cells[i][i] for i in range(size)) % 3 == 0)
        model.Add(sum(cells[i][size - i - 1] for i in range(size)) % 3 == 0)

    primes = {2, 3, 5, 7}
    for r in range(size):
        for c in range(size):
            if r < size - 1:
                model.AddForbiddenAssignments([cells[r][c], cells[r + 1][c]], [(p, p) for p in primes])
            if c < size - 1:
                model.AddForbiddenAssignments([cells[r][c], cells[r][c + 1]], [(p, p) for p in primes])

    solver = cp_model.CpSolver()
    if solver.Solve(model) == cp_model.FEASIBLE:
        return [[solver.Value(cells[r][c]) for c in range(size)] for r in range(size)]
    return None

sudoku_grid = [[0] * 9 for _ in range(9)]
solution = solve_sudoku(sudoku_grid)
if solution:
    for row in solution:
        print(row)

def define_csp_robot_navigation(grid, start, target):
    variables = [(r, c) for r in range(len(grid)) for c in range(len(grid[0]))]
    domains = {var: [(r + dr, c + dc) for dr, dc in [(-1, -1), (-1, 1), (1, -1), (1, 1)]
                      if 0 <= r + dr < len(grid) and 0 <= c + dc < len(grid[0]) and grid[r + dr][c + dc] != 1]
               for r, c in variables}
    constraints = []
    for (r, c), moves in domains.items():
        for move in moves:
            constraints.append(((r, c), move, ((r - move[0])**2 + (c - move[1])**2)**0.5))
    return variables, domains, constraints

In [ ]:
#TASK 2


from ortools.sat.python import cp_model

class IslandBoundarySolutionPrinter(cp_model.CpSolverSolutionCallback):
    def __init__(self, grid, boundary_vars):
        super().__init__()
        self.grid = grid
        self.boundary_vars = boundary_vars
        self.solution_found = False

    def on_solution_callback(self):
        if not self.solution_found:
            self.solution_found = True
            perimeter = sum(self.Value(var) for var in self.boundary_vars)
            print(f"Island Boundary Perimeter: {perimeter}")
            self.StopSearch()

def solve_island_perimeter(grid):
    model = cp_model.CpModel()
    rows, cols = len(grid), len(grid[0])

    # Variables: Boundary indicator (1 if part of boundary, 0 otherwise)
    boundary_vars = []

    for i in range(rows):
        for j in range(cols):
            if grid[i][j] == 1:  # Only land
                boundary_var = model.NewIntVar(0, 1, f"boundary_{i}_{j}")

                # Check adjacent cells (up, down, left, right)
                adjacent_water = []
                for di, dj in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
                    ni, nj = i + di, j + dj
                    if 0 <= ni < rows and 0 <= nj < cols and grid[ni][nj] == 0:
                        adjacent_water.append(1)  # Water detected

                # If any adjacent cell is water, mark as boundary
                if adjacent_water:
                    model.Add(boundary_var == 1)
                else:
                    model.Add(boundary_var == 0)

                boundary_vars.append(boundary_var)

    # Solve the CSP model
    solver = cp_model.CpSolver()
    solution_printer = IslandBoundarySolutionPrinter(grid, boundary_vars)

    print("Solving island boundary detection problem...")
    status = solver.Solve(model, solution_printer)

    if status in [cp_model.FEASIBLE, cp_model.OPTIMAL]:
        print("Solution found!")
    else:
        print("No solution found.")

# Sample island grid (5x5 example)
# 1 = Land, 0 = Water
island_grid = [
    [0, 1, 1, 1, 0],
    [1, 1, 1, 0, 0],
    [1, 1, 0, 0, 1],
    [0, 1, 1, 1, 1],
    [0, 0, 1, 1, 0]
]

# Solve for perimeter
solve_island_perimeter(island_grid)


In [ ]:
#TASK 3


from ortools.sat.python import cp_model
distances = [

    [0, 2, 9, 10, 5, 7, 6, 8, 4, 3],

    [2, 0, 8, 7, 6, 3, 5, 2, 9, 10],

    [9, 8, 0, 5, 6, 4, 3, 7, 2, 6],

    [10, 7, 5, 0, 4, 8, 9, 3, 2, 1],

    [5, 6, 6, 4, 0, 2, 8, 5, 7, 9],

    [7, 3, 4, 8, 2, 0, 3, 6, 9, 10],

    [6, 5, 3, 9, 8, 3, 0, 2, 4, 6],

    [8, 2, 7, 3, 5, 6, 2, 0, 9, 4],

    [4, 9, 2, 2, 7, 9, 4, 9, 0, 5],

    [3, 10, 6, 1, 9, 10, 6, 4, 5, 0]

]

def TSP():

    model = cp_model.CpModel()

    n = len(distances)

    city_position = [model.NewIntVar(0, n - 1, f'city_{i}') for i in range(n)]

    model.AddAllDifferent(city_position)



    total_distance = model.NewIntVar(0, sum(sum(row) for row in distances), 'total_distance')

    distance_expr = []

    for i in range(n):

        next_city = (i + 1) % n

        distance_expr.append(distances[city_position[i].Index()][city_position[next_city].Index()])

    model.Add(total_distance == sum(distance_expr))

    solver = cp_model.CpSolver()

    status = solver.Solve(model)

    if status == cp_model.FEASIBLE or status == cp_model.OPTIMAL:

        print("Optimal Tour Order:")

        for i in range(n):

            print(f"City {i}: visited at number {solver.Value(city_position[i])}")

        print(f"Total Distance: {solver.Value(total_distance)}")

    else:

        print("No solution found.")

TSP()



In [ ]:
# task 4


from ortools.sat.python import cp_model

class WarehouseCSP:
    def __init__(self, grid_size, num_robots, num_packages, capacities, weights, start_positions, destinations, charge_stations):
        self.grid_size = grid_size
        self.num_robots = num_robots
        self.num_packages = num_packages
        self.capacities = capacities
        self.weights = weights
        self.start_positions = start_positions
        self.destinations = destinations
        self.charge_stations = charge_stations
        self.model = cp_model.CpModel()
        self.create_variables()
        self.add_constraints()

    def create_variables(self):
        self.positions = {}
        self.assignments = {}
        for r in range(self.num_robots):
            for t in range(self.grid_size**2):  # Time steps
                self.positions[(r, t)] = self.model.NewIntVar(0, self.grid_size**2 - 1, f'pos_r{r}_t{t}')

        for p in range(self.num_packages):
            self.assignments[p] = self.model.NewIntVar(0, self.num_robots - 1, f'assign_p{p}')

    def add_constraints(self):
        for p in range(self.num_packages):
            robot = self.assignments[p]
            self.model.AddElement(robot, self.capacities) >= self.weights[p]

        for r in range(self.num_robots):
            for t in range(1, self.grid_size**2):
                prev_pos = self.positions[(r, t-1)]
                cur_pos = self.positions[(r, t)]
                self.model.AddAbs(cur_pos // self.grid_size - prev_pos // self.grid_size) + \
                self.model.AddAbs(cur_pos % self.grid_size - prev_pos % self.grid_size) <= 1  # Move only one step

    def solve(self):
        solver = cp_model.CpSolver()
        status = solver.Solve(self.model)
        if status == cp_model.FEASIBLE or status == cp_model.OPTIMAL:
            for r in range(self.num_robots):
                print(f'Robot {r} path: ', [solver.Value(self.positions[(r, t)]) for t in range(self.grid_size**2)])
            for p in range(self.num_packages):
                print(f'Package {p} assigned to robot {solver.Value(self.assignments[p])}')
        else:
            print("No solution found")

if __name__ == "__main__":
    grid_size = 6
    num_robots = 5
    num_packages = 10
    capacities = [10, 15, 12, 20, 8]
    weights = [2, 4, 5, 3, 7, 6, 8, 2, 3, 4]
    start_positions = [0, 5, 10, 15, 20]
    destinations = [35, 30, 25, 20, 15, 10, 5, 0, 3, 6]
    charge_stations = [1, 9, 18, 27, 36]

    warehouse = WarehouseCSP(grid_size, num_robots, num_packages, capacities, weights, start_positions, destinations, charge_stations)
    warehouse.solve()

In [ ]:
# task 5


from ortools.sat.python import cp_model

class SudokuCSP:
    def __init__(self, grid):
        self.grid = grid
        self.model = cp_model.CpModel()
        self.variables = {}
        self.create_variables()
        self.add_constraints()

    def create_variables(self):
        for i in range(9):
            for j in range(9):
                if self.grid[i][j] == 0:
                    self.variables[(i, j)] = self.model.NewIntVar(1, 9, f'cell_{i}_{j}')
                else:
                    self.variables[(i, j)] = self.model.NewIntVarFromDomain(cp_model.Domain.FromValues([self.grid[i][j]]), f'cell_{i}_{j}')

    def add_constraints(self):
        for i in range(9):
            self.model.AddAllDifferent([self.variables[(i, j)] for j in range(9)])  # Row constraint
            self.model.AddAllDifferent([self.variables[(j, i)] for j in range(9)])  # Column constraint

        for i in range(3):
            for j in range(3):
                self.model.AddAllDifferent([self.variables[(x, y)] for x in range(i*3, i*3+3) for y in range(j*3, j*3+3)])

        self.model.Add(sum(self.variables[(i, i)] for i in range(9)) % 3 == 0)  # Main diagonal constraint
        self.model.Add(sum(self.variables[(i, 8 - i)] for i in range(9)) % 3 == 0)  # Anti-diagonal constraint

        primes = {2, 3, 5, 7}
        for i in range(9):
            for j in range(9):
                if (i, j) in self.variables:
                    cell = self.variables[(i, j)]
                    for di, dj in [(0, 1), (1, 0)]:
                        ni, nj = i + di, j + dj
                        if (ni, nj) in self.variables:
                            neighbor = self.variables[(ni, nj)]
                            self.model.AddForbiddenAssignments([cell, neighbor], [(p, p2) for p in primes for p2 in primes])

    def solve(self):
        solver = cp_model.CpSolver()
        status = solver.Solve(self.model)
        if status in [cp_model.FEASIBLE, cp_model.OPTIMAL]:
            for i in range(9):
                print([solver.Value(self.variables[(i, j)]) for j in range(9)])
        else:
            print("No solution found")

if __name__ == "__main__":
    sudoku_grid = [
        [5, 3, 0, 0, 7, 0, 0, 0, 0],
        [6, 0, 0, 1, 9, 5, 0, 0, 0],
        [0, 9, 8, 0, 0, 0, 0, 6, 0],
        [8, 0, 0, 0, 6, 0, 0, 0, 3],
        [4, 0, 0, 8, 0, 3, 0, 0, 1],
        [7, 0, 0, 0, 2, 0, 0, 0, 6],
        [0, 6, 0, 0, 0, 0, 2, 8, 0],
        [0, 0, 0, 4, 1, 9, 0, 0, 5],
        [0, 0, 0, 0, 8, 0, 0, 7, 9]
    ]
    sudoku = SudokuCSP(sudoku_grid)
    sudoku.solve()